In [5]:
import plotly.graph_objects as go

# 노드 정의 (이름과 좌표 x, y)
# particle(0,2) / gas(-1,1), pressure(0,1), temp(1,1) / h2(-1.5,0), nf3(-0.5,0)
x_nodes = [0, -1, 0, 1, -1.5, -0.5]
y_nodes = [2, 1, 1, 1, 0, 0]
node_names = ["particle", "gas", "pressure", "temp", "h2", "nf3"]

# 엣지(선) 정의 (연결할 좌표 쌍)
# (particle -> gas), (particle -> pressure), (particle -> temp)
# (gas -> h2), (gas -> nf3)
x_edges = [0, -1, None, 0, 0, None, 0, 1, None, -1, -1.5, None, -1, -0.5]
y_edges = [2, 1, None, 2, 1, None, 2, 1, None, 1, 0, None, 1, 0]

fig = go.Figure()

# 선 그리기
fig.add_trace(go.Scatter(
    x=x_edges, y=y_edges,
    mode='lines',
    line=dict(color='black', width=1),
    hoverinfo='none'
))

# 점(노드) 그리기
fig.add_trace(go.Scatter(
    x=x_nodes, y=y_nodes,
    mode='markers+text',
    text=node_names,
    textposition="bottom center",
    marker=dict(size=20, color='skyblue', line=dict(color='black', width=1)),
    hoverinfo='text'
))

fig.update_layout(
    title="Particle Tree Diagram",
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

fig.show()

In [6]:
fig.write_html('test.html')

In [19]:
import plotly.graph_objects as go
import pandas as pd 
data = pd.DataFrame()
data['sensor'] = ['top temp', 'bottom temp', 'h2', 'nf3', 'pressure', 'throttle valve']
data.index=data['sensor']
data['group']= 'temp'
data.loc['h2','group'] = 'gas'
data.loc['nf3','group'] = 'gas'
data.loc['pressure','group'] = 'press'
data.loc['throttle valve','group'] = 'press'
data.to_csv('data.csv')

In [8]:

labels = ["particle", "gas", "pressure", "temp", "h2", "nf3"]
parents = ["", "particle", "particle", "particle", "gas", "gas"]

fig = go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    branchvalues="total"  # 하위 항목의 크기 합이 부모와 같게 (데이터가 있을 경우 유용)
))

fig.update_layout(
    title="Particle Hierarchy Structure (Sunburst)",
    margin=dict(t=50, l=25, r=25, b=25)
)

fig.write_html('test.html')

In [21]:
import pandas as pd
import json

# 1. 데이터 정의 (Pandas)
data = pd.DataFrame()
data['sensor'] = ['top temp', 'bottom temp', 'h2', 'nf3', 'pressure', 'throttle valve']
data.index = data['sensor']
data['group'] = 'temp'
data.loc['h2','group'] = 'gas'
data.loc['nf3','group'] = 'gas'
data.loc['pressure','group'] = 'press'
data.loc['throttle valve','group'] = 'press'

# 2. JSON 변환 (DataFrame -> D3 JSON)
groups = data.groupby('group')
children_list = []

for group_name, group_df in groups:
    sensors = []
    for _, row in group_df.iterrows():
        sensors.append({"name": row['sensor']})
    
    children_list.append({
        "name": group_name,
        "children": sensors
    })

tree_structure = {
    "name": "particle",
    "children": children_list
}

json_data = json.dumps(tree_structure, indent=2)

# 3. HTML 생성 (Highlight 로직 추가됨)
html_content = f"""
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>D3.js Highlight Path</title>
    <script src="./static/d3.v7.min.js"></script>
    <style>
        .node circle {{
            fill: #fff;
            stroke: steelblue;
            stroke-width: 3px;
        }}
        .node text {{
            font: 14px sans-serif;
            font-weight: bold;
        }}
        /* 기본 링크 스타일은 여기서 제거하고 JS에서 제어합니다 */
        .link {{
            fill: none;
        }}
    </style>
</head>
<body>

<h3>Sensor Hierarchy (Red path to 'h2')</h3>
<div id="tree-container"></div>

<script>
    const treeData = {json_data};

    // 캔버스 설정
    const margin = {{top: 40, right: 120, bottom: 50, left: 120}},
          width = 800 - margin.left - margin.right,
          height = 500 - margin.top - margin.bottom;

    const svg = d3.select("#tree-container").append("svg")
        .attr("width", width + margin.left + margin.right)
        .attr("height", height + margin.top + margin.bottom)
        .append("g")
        .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

    const treemap = d3.tree().size([height, width]);
    let root = d3.hierarchy(treeData, d => d.children);
    root.x0 = height / 2;
    root.y0 = 0;

    const treeNodes = treemap(root);

    // ---------------------------------------------------------
    // [핵심 로직] 'h2' 노드를 찾고, 루트까지 경로(Path) 추적하기
    // ---------------------------------------------------------
    const targetName = "h2"; // 찾고 싶은 하위 노드 이름
    let targetNode = root.descendants().find(d => d.data.name === targetName);
    
    // 경로에 포함된 노드들의 ID(또는 객체)를 Set에 저장
    let pathSet = new Set();
    while (targetNode) {{
        pathSet.add(targetNode);
        targetNode = targetNode.parent; // 부모로 이동
    }}

    // ---------------------------------------------------------
    // 링크 그리기 (조건부 스타일 적용)
    // ---------------------------------------------------------
    svg.selectAll(".link")
        .data(treeNodes.descendants().slice(1))
        .enter().append("path")
        .attr("class", "link")
        .attr("d", d => {{
            return "M" + d.y + "," + d.x
                 + "C" + (d.y + d.parent.y) / 2 + "," + d.x
                 + " " + (d.y + d.parent.y) / 2 + "," + d.parent.x
                 + " " + d.parent.y + "," + d.parent.x;
        }})
        // [스타일 변경] 경로에 포함된 링크면 빨간색 + 두껍게
        .attr("stroke", d => pathSet.has(d) ? "red" : "#ccc")
        .attr("stroke-width", d => pathSet.has(d) ? "4px" : "2px")
        .attr("stroke-opacity", d => pathSet.has(d) ? 1 : 0.5); // 나머지는 약간 흐리게

    // 노드 그리기
    const node = svg.selectAll(".node")
        .data(treeNodes.descendants())
        .enter().append("g")
        .attr("class", "node")
        .attr("transform", d => "translate(" + d.y + "," + d.x + ")");

    node.append("circle")
        .attr("r", 10)
        // [선택 사항] 경로상의 노드 테두리도 빨간색으로 할지 여부
        .style("stroke", d => pathSet.has(d) ? "red" : "steelblue");

    node.append("text")
        .attr("dy", ".35em")
        .attr("x", d => d.children ? -13 : 13)
        .style("text-anchor", d => d.children ? "end" : "start")
        .text(d => d.data.name);

</script>
</body>
</html>
"""

with open("tree_highlight.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("파일 생성 완료: tree_highlight.html")

파일 생성 완료: tree_highlight.html
